In [1]:
# Torch imports
from torch.utils.data import DataLoader

# Custom imports
import torch
import heuristics
from dataset_utils import PathPlanningDataset
from classic_astar import Astar
from training_module import NeuralAstarTrainingModule
from neural_astar import NeuralAstar
import lightning as pl
from differentiable_astar import DifferentiableAstar
import encoders_refactored

In [2]:
# Dataset Generation

initial_dataset = PathPlanningDataset(
    "/workspaces/neural-Astar/datasets/raw/street",
    32,
    5,
    heuristics.euclidian,
    randomize_points=False,
    max_astar_iterations=10000,
)

training_dataloader = DataLoader(
    initial_dataset, batch_size=10, shuffle=True, drop_last=True, num_workers=7
)  # TODO fix for batch_size = 1

Path not found, skipping:  Path not found
Path not found, skipping:  Path not found
Path not found, skipping:  Path not found
Path not found, skipping:  Path not found


In [3]:
# Create Astar instances
classic_astar = Astar(
    heuristic=heuristics.euclidian, costmap_weight=5, heuristic_weight=5
)
differentiable_astar = DifferentiableAstar(max_iterations=5000, costmap_weight=10)

In [4]:
import segmentation_models_pytorch

unet = segmentation_models_pytorch.Unet(
    encoder_name="resnet18",
    encoder_depth=3,
    encoder_weights=None,
    decoder_channels=[2, 4, 8],
    in_channels=2,
    classes=1,
)

cnn = encoders_refactored.CNN(2, 4)
astar_encoder = encoders_refactored.AstarEncoder()
astar_encoder.add_encoder(cnn)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
neural_astar = NeuralAstar(
    classic_astar=classic_astar,
    differentiable_astar=differentiable_astar,
    encoder=astar_encoder,
    use_start_goal_data=True,
)

In [6]:
astar_training_module = NeuralAstarTrainingModule(neural_astar=neural_astar)

In [7]:
trainer = pl.Trainer(
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    log_every_n_steps=1,
    default_root_dir="../logs/",
    max_epochs=500,
)

GPU available: False, used: False


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [8]:
trainer.fit(astar_training_module, training_dataloader)


  | Name          | Type        | Params
----------------------------------------------
0 | neural_astar_ | NeuralAstar | 391 K 
----------------------------------------------
391 K     Trainable params
9         Non-trainable params
391 K     Total params
1.566     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/44 [00:00<?, ?it/s] 

/opt/conda/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch 3:  16%|█▌        | 7/44 [00:05<00:27,  1.37it/s, v_num=2, train_loss=0.299] 

/opt/conda/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [1]:
from display import Displayer
import matplotlib.pyplot as plt

map_batch, start_batch, goal_batch, path_batch = next(iter(training_dataloader))
map_batch = map_batch.unsqueeze(1)
start_batch = start_batch.unsqueeze(1)
goal_batch = goal_batch.unsqueeze(1)


print(map_batch.shape)

cosmtap_batch = neural_astar._get_cost_batch(map_batch, start_batch + goal_batch)
cosmtap_batch = cosmtap_batch.detach()
out_path, out_searched = neural_astar(map_batch, start_batch, goal_batch)

d = Displayer()
d.draw_full_astar_output(
    map_batch[0, 0],
    start_batch[0, 0],
    goal_batch[0, 0],
    out_searched[0, 0],
    out_path[0, 0],
    cosmtap_batch[0, 0],
)

NameError: name 'training_dataloader' is not defined